In [1]:
import re
from collections import Counter
import json
import os
import pandas as pd
import pickle


def format_data(task_name, path, to_folfer, row_len, has_b=False):
    res = []
    with open(path, "r") as f:
        for line in f:
            s = re.split(r"[\t\n]", line)
            s = s[:-1]
            if len(s) != row_len:
                print("format error", line)
                continue
            tmp = {}
            tmp["id"] = s[0]
            tmp["text_a"] = s[1]
            tmp["text_b"] = s[2] if has_b else None
            tmp["label"] = s[3] if has_b else s[2]
            tmp["task"] = task_name
            res.append(tmp)
    print(">>>>>>>>>>>>>>>>>>>>基本统计<<<<<<<<<<<<<<<<<<<<<<<")
    print("总数", len(res))
    print("最后一行", res[-1])
    with open(os.path.join(to_folfer, task_name + "_samples.json"), "w") as f:
        json.dump(res, f, ensure_ascii=False, indent=2)

    label_fq = Counter([i["label"] for i in res])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>label统计<<<<<<<<<<<<<<<<<<<<<<<<<")
    print(label_fq)

    label2index = {}
    index2label = {}
    for index, label in enumerate(label_fq.keys()):
        label2index[label] = index
        index2label[index] = label
    print("label2index", label2index)
    print("index2label", index2label)
    with open(os.path.join(to_folfer, task_name + "_label2index.pkl"), "wb") as f:
        pickle.dump(label2index, f)
    with open(os.path.join(to_folfer, task_name + "_index2label.pkl"), "wb") as f:
        pickle.dump(index2label, f)

    df_len = pd.DataFrame(
        sorted([len(t["text_a"]) + len(t["text_b"]) if has_b else len(t["text_a"]) for t in res], reverse=True))
    print(">>>>>>>>>>>>>>>>>>>>>>>>句子长 统计<<<<<<<<<<<<<<<<<<<<<<<<")
    print(df_len.describe())
    print("<125占比", len(df_len[df_len[0] < 128 - 3]) / len(df_len))
    print("<253占比", len(df_len[df_len[0] < 256 - 3]) / len(df_len))


print("绝对路径==")
print(os.path.abspath("OCEMOTION_train1128.csv"))
print(os.path.abspath("."))
print(os.getcwd())

format_data("oce", "../src/data/OCEMOTION_train1128.csv", "../src/data/", row_len=3, has_b=False)
format_data("ocn", "../src/data/OCNLI_train1128.csv", "../src/data/", row_len=4, has_b=True)
format_data("tn", "../src/data/TNEWS_train1128.csv", "../src/data/", row_len=3, has_b=False)

绝对路径==
/Users/qianlai/PycharmProjects/tcfhbs/nb/OCEMOTION_train1128.csv
/Users/qianlai/PycharmProjects/tcfhbs/nb
/Users/qianlai/PycharmProjects/tcfhbs/nb
>>>>>>>>>>>>>>>>>>>>基本统计<<<<<<<<<<<<<<<<<<<<<<<
总数 35694
最后一行 {'id': '35693', 'text_a': '听说你最近很上进哦,哈!', 'text_b': None, 'label': 'happiness', 'task': 'oce'}
>>>>>>>>>>>>>>>>>>>>>>>>>>>label统计<<<<<<<<<<<<<<<<<<<<<<<<<
Counter({'sadness': 12604, 'happiness': 8981, 'disgust': 4405, 'anger': 4115, 'like': 4087, 'surprise': 908, 'fear': 594})
label2index {'sadness': 0, 'happiness': 1, 'like': 2, 'anger': 3, 'fear': 4, 'surprise': 5, 'disgust': 6}
index2label {0: 'sadness', 1: 'happiness', 2: 'like', 3: 'anger', 4: 'fear', 5: 'surprise', 6: 'disgust'}
>>>>>>>>>>>>>>>>>>>>>>>>句子长 统计<<<<<<<<<<<<<<<<<<<<<<<<
                  0
count  35694.000000
mean      47.539671
std       38.881512
min        3.000000
25%       18.000000
50%       34.000000
75%       67.000000
max      309.000000
<125占比 0.9193982181879308
<253占比 0.9999439681739228
>>>>>>>